<a href="https://colab.research.google.com/github/carlosbarrone/public_notebooks/blob/dev/network_analysis_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Project constants
WORKING_DIR = '/content/drive/MyDrive/Colab Notebooks/msc_ds_boulder/text_mkt_anlysis_specialization/network_analysis'
TOTAL_TWEETS = 175078

In [2]:
# Imports
from google.colab import drive

In [3]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
requirements = ['pyvis', 'python-twitter', 'tqdm']
try:
  with open(f'./requirements.txt','r') as f:
    !pip install -r ./requirements.txt
except FileNotFoundError as fnf:
  with open(f'./requirements.txt', 'a') as f:
    for r in requirements:
      f.write(f'{r}\n')
  !pip install -r ./requirements.txt
except Exception as e:
  print(e)

In [5]:
# Imports
import glob
import os
import shutil
import json
import csv
import networkx as nx
import matplotlib.pyplot as plt
import pyvis
import gzip
import json
import pprint

import pandas as pd

from pyvis.network import Network
from time import sleep
from twitter.models import Status
from tqdm import tqdm

In [6]:
pp = pprint.PrettyPrinter(indent=4)

with gzip.open(f'{WORKING_DIR}/nike_lululemon_adidas_tweets.jsonl.gz') as f:
  d = json.loads(f.readline())
  pp.pprint(d)

{   'contributors': None,
    'coordinates': None,
    'created_at': 'Fri Oct 01 08:25:03 +0000 2021',
    'display_text_range': [0, 146],
    'entities': {   'hashtags': [   {'indices': [0, 3], 'text': 'ad'},
                                    {   'indices': [102, 116],
                                        'text': 'SneakerScouts'}],
                    'media': [   {   'display_url': 'pic.twitter.com/wmjxIcsheP',
                                     'expanded_url': 'https://twitter.com/SneakerScouts/status/1443854459625431041/photo/1',
                                     'id': 1443854458404941800,
                                     'id_str': '1443854458404941827',
                                     'indices': [147, 170],
                                     'media_url': 'http://pbs.twimg.com/media/FAmZ-uSXMAMy-iz.png',
                                     'media_url_https': 'https://pbs.twimg.com/media/FAmZ-uSXMAMy-iz.png',
                                     'sizes': {   'l

In [76]:
def tweak_tweets_df(df:pd.DataFrame)->pd.DataFrame:
  return df.drop(columns=['user_mentions', 'user_mentions_ids'])

In [77]:
try:
  del tweets
except NameError as ne:
  print(ne)

name 'tweets' is not defined


In [78]:
tweets = []
with gzip.open(f'{WORKING_DIR}/nike_lululemon_adidas_tweets.jsonl.gz') as f:
  for l in tqdm(f, total=TOTAL_TWEETS):
    t = json.loads(l)
    s = Status().NewFromJsonDict(t)
    tweet = {
        'tweet_id': s.id,
        'tweet_user_id': s.user.id,
        'tweet_user_id_str': s.user.id_str,
        'tweet_user_screen_name': s.user.screen_name,
        'tweet_user_followers_count': s.user.followers_count,
        'tweet_text': s.full_text
    }
    user_mentions_ids = []
    user_mentions_ids_str = []
    user_mentions_screen_names = []
    user_mentions = []
    if 'entities' in t:
      if 'user_mentions' in t['entities']:
        for um in t['entities']['user_mentions']:
          user_mentions_ids.append(um['id'])
          user_mentions_ids_str.append(um['id_str'])
          user_mentions_screen_names.append(um['screen_name'])
          user_mentions.append((um['id'], um['id_str'], um['screen_name']))
        tweet['user_mentions_ids'] = user_mentions_ids
        tweet['user_mentions_ids_str'] = user_mentions_ids_str
        tweet['user_mentions_screen_names'] = user_mentions_screen_names
        tweet['user_mentions'] = user_mentions
    tweets.append(tweet)

100%|██████████| 175078/175078 [01:01<00:00, 2861.30it/s]


In [79]:
tweets_df = tweak_tweets_df(pd.DataFrame(tweets))
tweets_df.sample(3)

,tweet_id,tweet_user_id,tweet_user_id_str,tweet_user_screen_name,tweet_user_followers_count,tweet_text,user_mentions_ids_str,user_mentions_screen_names
127257,1467997583469600800,337649347,337649347,KVSwitzer,9135,Breaking News! Who wants to support women by j...,"[2400458677, 111037335, 82730603, 20434358, 30...","[261Fearless, bostonmarathon, adidasWomen, adi..."
111973,1464620311228338200,773885035938218000,773885035938217984,amcmath04,368,@Kaya_Alexander5 @Nike Thanks 😊,"[989588511941578752, 415859364]","[Kaya_Alexander5, Nike]"
133918,1469047329143632000,129078781,129078781,boujayyyyy,1348,RT @theestallion: I’m officially a @Nike YARDR...,"[104040220, 415859364]","[theestallion, Nike]"


In [80]:
len(tweets_df)

175078

In [81]:
del tweets

In [82]:
len(tweets_df.tweet_user_id_str.unique())

104772

In [83]:
twitter_users = pd.DataFrame(
    tweets_df[['tweet_user_id_str','tweet_user_screen_name','tweet_user_followers_count']].sort_values(by=['tweet_user_followers_count']).drop_duplicates(['tweet_user_id_str'], keep='last'),
    columns=['tweet_user_id_str','tweet_user_screen_name','tweet_user_followers_count']
)
twitter_users

,tweet_user_id_str,tweet_user_screen_name,tweet_user_followers_count
102077,791158105,soni_sumir,0
37133,2519804105,xinatus,0
170091,1391269849356111875,ICOutlook,0
97727,1442164147173543944,Kyzan123,0
96521,816347651771953152,OminousWatcher,0
...,...,...,...
121270,932429090,MarceloM12,12258139
160195,24742040,Xbox,17391710
27967,19583545,LFC,18891253
150062,19923144,NBA,35763024


In [84]:
twitter_users = twitter_users.join(
    tweets_df[['tweet_user_id_str','tweet_id']].groupby('tweet_user_id_str').count(),
    on='tweet_user_id_str',
    how='inner'
).rename(columns={'tweet_id':'tweet_count'})

In [85]:
print(len(twitter_users))
twitter_users.sample(3)

104772


,tweet_user_id_str,tweet_user_screen_name,tweet_user_followers_count,tweet_count
39857,154197491,LennyMiami,929,2
47014,584519061,doogi3ville,1472,1
170097,1465520504878645252,demore_joanna,133,2


In [110]:
user_subset = twitter_users.query('tweet_user_followers_count > 100000 and tweet_count > 1')
user_subset_list = user_subset.tweet_user_id_str.to_list()
print(len(user_subset))
user_subset.sample(3)

196


,tweet_user_id_str,tweet_user_screen_name,tweet_user_followers_count,tweet_count
149592,167421802,BurgerKing,1972152,4
173929,861700170,bajabiri,252529,10
98460,18949998,atmos_usa,105936,3


In [87]:
print(len(user_subset)/len(twitter_users))

0.0018707288206772801


In [108]:
Graph = nx.DiGraph()
Graph.clear()

In [89]:
tweets_df.sample(2)

,tweet_id,tweet_user_id,tweet_user_id_str,tweet_user_screen_name,tweet_user_followers_count,tweet_text,user_mentions_ids_str,user_mentions_screen_names
87471,1459222943591022600,1404474791759855600,1404474791759855618,peezus_christ,248,RT @EASPORTSFIFA: X x Predator x Copa\n\nWhich...,"[105297123, 300114634]","[EASPORTSFIFA, adidas]"
31395,1450139094965792800,731982306764689400,731982306764689408,Batman1El,9,"RT @Xbox: Three icons: @adidas, Xbox, and you....","[24742040, 300114634]","[Xbox, adidas]"


In [117]:
tweets_df[tweets_df.tweet_user_id_str.isin(user_subset_list)].apply(lambda r: [Graph.add_edge(r['tweet_user_screen_name'],u) for u in r['user_mentions_screen_names']], axis=1)

87                                       [None, None, None]
734                                                  [None]
1370                   [None, None, None, None, None, None]
1567                   [None, None, None, None, None, None]
1577                         [None, None, None, None, None]
                                ...                        
174495    [None, None, None, None, None, None, None, Non...
174548                                         [None, None]
174556     [None, None, None, None, None, None, None, None]
174618           [None, None, None, None, None, None, None]
174776                                               [None]
Length: 906, dtype: object

In [118]:
f'Our Graphs has {Graph.number_of_nodes()} nodes and {Graph.number_of_edges()} edges.'

'Our Graphs has 872 nodes and 1178 edges.'